In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from utils import process_authors

In [2]:
data = pd.read_csv('../clustering/results/004_of_V13_topics.csv')

data.drop(data[data['authors'].map(lambda x: '\\' in r'%r' % x)].index, inplace=True)
data['authors'] = data['authors'].apply(lambda x: json.loads(x.replace('\'', '\"')))
data['authors'] = data['authors'].apply(lambda x:  process_authors(x))
data.drop(data[data['authors'].map(lambda x: x == '[]')].index, inplace=True)

#### Perform train test split before converting data to graph format

In [3]:
train, test = train_test_split(data, test_size=0.2)
train.shape, test.shape

((42412, 24), (10604, 24))

In [4]:
train, val = train_test_split(train, test_size=0.2)
train.shape, val.shape

((33929, 24), (8483, 24))

#### Process data to format sutable for LightGCN

In [5]:
import itertools

def create_lightgcn_data(data, type, num_authors=0):
    unique_authors = set()
    edges = []
    for i in range(len(data)):
        author_names = set()
        for author in data.iloc[i]['authors']:
            if 'name' in author:
                author_names.add(author['name'])
        edges.append(list(itertools.product(list(author_names), list(author_names))))
        unique_authors = unique_authors.union(author_names)

    unique_authors_id = {}
    with open(f'data/authors_{type}.txt', 'w') as f:
        for i, author in enumerate(unique_authors):
            unique_authors_id[author] = i + num_authors
            f.write(f'{i + num_authors} {author}\n')
        
    with open(f'data/aminer-rec/raw/{type}.txt', 'w') as f:
        for edge in edges:
            for pair in edge:
                if pair[0] != pair[1]:
                    f.write(f'{unique_authors_id[pair[0]]} {unique_authors_id[pair[1]]}\n')
    
    return len(unique_authors)

In [6]:
train_num = create_lightgcn_data(train, 'train')

In [7]:
val_num = create_lightgcn_data(val, 'valid', train_num)

In [8]:
test_num = create_lightgcn_data(test, 'test', train_num + val_num)

In [10]:
!cat data/authors_train.txt >> data/authors_id.txt
!cat data/authors_valid.txt >> data/authors_id.txt
!cat data/authors_test.txt >> data/authors_id.txt